In [17]:
!pip install xlrd
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [ ]:
df=pd.read_excel("news.xls")
df.head()

In [ ]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['category'])

In [ ]:
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['content'])
sequences = tokenizer.texts_to_sequences(df['content'])

In [ ]:
max_len = 15
padded = pad_sequences(sequences, maxlen=max_len, padding='post')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded, df['label'].values, test_size=0.25, random_state=42)


In [ ]:
model = Sequential([
    Embedding(input_dim=1000, output_dim=64, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=False, dropout=0.2, recurrent_dropout=0.2)),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=4,
                    validation_data=(X_test, y_test))

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test doğruluğu: {accuracy:.2f}")

In [ ]:
sample_text = ["Yeni futbol sezonu başladı"]
seq = tokenizer.texts_to_sequences(sample_text)
padded_seq = pad_sequences(seq, maxlen=max_len, padding='post')
pred = model.predict(padded_seq)
pred_label = label_encoder.inverse_transform([np.argmax(pred)])
print("Tahmin edilen kategori:", pred_label[0])